In [40]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

In [41]:
torch.manual_seed(0)
np.random.seed(0)

In [42]:
df = pd.read_csv("/Users/michaelhein/Programs/Dicebot/outputs/output.txt")

In [43]:
df.head()

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
0,25,Agg-High,5,9,4,2,0,0,3,0,0
1,24,Agg-Low,5,8,4,0,0,1,2,2,0
2,24,Cond-Low,5,9,2,1,2,1,0,0,1
3,23,Cond-High,5,9,3,1,0,3,0,1,0
4,22,Conserv,4,1,4,0,1,0,2,0,1


In [44]:
df["totalDice"]

0         25
1         24
2         24
3         23
4         22
          ..
590880     6
590881     6
590882     5
590883     4
590884     4
Name: totalDice, Length: 590885, dtype: int64

In [45]:
df.shape

(590885, 11)

In [46]:
df[["totalDice", "player", "numPlayerDice", "quantityDeclared", "diceNumDeclared"]]

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared
0,25,Agg-High,5,9,4
1,24,Agg-Low,5,8,4
2,24,Cond-Low,5,9,2
3,23,Cond-High,5,9,3
4,22,Conserv,4,1,4
...,...,...,...,...,...
590880,6,Agg-High,1,2,5
590881,6,Agg-Low,1,3,4
590882,5,Cond-High,3,2,3
590883,4,Agg-Low,1,1,4


In [47]:
features = df[df.columns[:5]]
features.head()

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared
0,25,Agg-High,5,9,4
1,24,Agg-Low,5,8,4
2,24,Cond-Low,5,9,2
3,23,Cond-High,5,9,3
4,22,Conserv,4,1,4


In [48]:
targets = df[df.columns[5:]]
targets.head()

,1s,2s,3s,4s,5s,6s
0,2,0,0,3,0,0
1,0,0,1,2,2,0
2,1,2,1,0,0,1
3,1,0,3,0,1,0
4,0,1,0,2,0,1


In [49]:
AggHigh = df[df["player"] == "Agg-High"]
AggHigh = AggHigh.drop(columns = ["player"])
AggHigh.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
0,25,5,9,4,2,0,0,3,0,0
5,22,4,7,2,1,1,0,1,1,0
10,21,4,7,2,0,2,0,0,1,1
15,19,4,6,1,1,1,0,0,1,1
20,18,4,5,4,0,0,1,1,1,1


In [50]:
td, test = train_test_split(AggHigh, test_size=0.1)

In [51]:
train, develop = train_test_split(AggHigh, test_size=0.2)

In [52]:
train.shape

(86946, 10)

In [53]:
develop.shape

(21737, 10)

In [54]:
test.shape

(10869, 10)

In [55]:
x_train = train[train.columns[:4]]
y_train = train[train.columns[4:]]

x_develop = develop[develop.columns[:4]]
y_develop = develop[develop.columns[4:]]

x_test = test[test.columns[:4]]
y_test = test[test.columns[4:]]

In [56]:
x_train_tensor = torch.from_numpy(x_train.to_numpy()).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).float()

x_develop_tensor = torch.from_numpy(x_develop.to_numpy()).float()
y_develop_tensor = torch.from_numpy(y_develop.to_numpy()).float()

x_test_tensor = torch.from_numpy(x_test.to_numpy()).float()
y_test_tensor = torch.from_numpy(y_test.to_numpy()).float()

In [57]:
train_dataset = list(zip(x_train_tensor, y_train_tensor))
develop_dataset = list(zip(x_develop_tensor, y_develop_tensor))
test_dataset = list(zip(x_test_tensor, y_test_tensor))

In [58]:
class Net(nn.Module):
    def __init__(self, num_layers, layers_size):
        super(Net, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(4, layers_size)])
        self.hidden_layers.extend([nn.Linear(layers_size, layers_size) for i in range(1, num_layers-1)])
        self.output = nn.Linear(layers_size, 6)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        z = x
        for layer in self.hidden_layers:
            z = self.activation(layer(z))
        return self.output(z)

In [59]:
class Trainer():
    def __init__(self, net, optim, loss_function, train_loader):
        self.net = net
        self.optim = optim
        self.loss_function = loss_function
        self.train_loader = train_loader
        
    def train(self, epochs, verbose=True):
        losses = []
        for epoch in range(epochs):
            epoch_loss = 0.0
            epoch_steps = 0
            for data in self.train_loader:
                x = data[0]
                y = data[1]
                
                self.optim.zero_grad()
                
                preds = self.net.forward(x)
                
                loss = self.loss_function(preds, y)
                
                loss.backward()
                
                self.optim.step()
                
                epoch_loss += loss.item()
                epoch_steps += 1
                
            losses.append(epoch_loss / epoch_steps)
            if verbose: print("epoch [%d]: loss %.3f" % (epoch+1, losses[-1]))
        return losses

In [60]:
## Hyperparamters

num_epochs = 30
batch_size = 256
learning_rate = 0.02

In [61]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
develop_loader = DataLoader(dataset=develop_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [62]:
net = Net(2, 16)
opt = optim.Adam(net.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()

trainer = Trainer(net=net, optim=opt, loss_function=loss_function, train_loader=train_loader)

losses = trainer.train(num_epochs)

epoch [1]: loss 0.398
epoch [2]: loss 0.346
epoch [3]: loss 0.319
epoch [4]: loss 0.314
epoch [5]: loss 0.310
epoch [6]: loss 0.307
epoch [7]: loss 0.304
epoch [8]: loss 0.302
epoch [9]: loss 0.302
epoch [10]: loss 0.299
epoch [11]: loss 0.294
epoch [12]: loss 0.290
epoch [13]: loss 0.289
epoch [14]: loss 0.283
epoch [15]: loss 0.284
epoch [16]: loss 0.281
epoch [17]: loss 0.281
epoch [18]: loss 0.281
epoch [19]: loss 0.279
epoch [20]: loss 0.279
epoch [21]: loss 0.280
epoch [22]: loss 0.278
epoch [23]: loss 0.280
epoch [24]: loss 0.277
epoch [25]: loss 0.280
epoch [26]: loss 0.280
epoch [27]: loss 0.279
epoch [28]: loss 0.278
epoch [29]: loss 0.278
epoch [30]: loss 0.278


In [63]:
AggHigh.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
0,25,5,9,4,2,0,0,3,0,0
5,22,4,7,2,1,1,0,1,1,0
10,21,4,7,2,0,2,0,0,1,1
15,19,4,6,1,1,1,0,0,1,1
20,18,4,5,4,0,0,1,1,1,1


In [64]:
train_dataset[0]

(tensor([6., 2., 3., 4.]), tensor([0., 0., 1., 1., 0., 0.]))

In [65]:
train_dataset[1]

(tensor([20.,  4.,  7.,  5.]), tensor([1., 0., 1., 0., 1., 1.]))

In [66]:
x_train.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared
555420,6,2,3,4
537425,20,4,7,5
201910,12,2,4,1
138204,6,3,2,2
495532,12,3,4,5


In [67]:
net.forward(train_dataset[0][0])

tensor([0.1379, 0.1169, 0.1522, 1.2482, 0.2216, 0.0786],
       grad_fn=<AddBackward0>)

In [68]:
net.forward(train_dataset[1][0])

tensor([0.3816, 0.5032, 0.3595, 0.7331, 1.1521, 0.8132],
       grad_fn=<AddBackward0>)

In [69]:
develop_dataset[0]

(tensor([8., 1., 3., 2.]), tensor([0., 1., 0., 0., 0., 0.]))

In [70]:
develop_dataset[1]

(tensor([17.,  4.,  7.,  4.]), tensor([0., 1., 0., 3., 0., 0.]))

In [71]:
develop_dataset[2]

(tensor([12.,  2.,  5.,  4.]), tensor([0., 0., 0., 1., 0., 1.]))

In [72]:
develop_dataset[3]

(tensor([23.,  5.,  8.,  3.]), tensor([1., 0., 2., 1., 0., 1.]))

In [73]:
net.forward(develop_dataset[0][0])

tensor([-0.0243,  1.1448, -0.0244,  0.0071, -0.1621,  0.0677],
       grad_fn=<AddBackward0>)

In [74]:
net.forward(develop_dataset[1][0])

tensor([0.4426, 0.5174, 0.4634, 1.5529, 0.6537, 0.3269],
       grad_fn=<AddBackward0>)

In [75]:
net.forward(develop_dataset[2][0])

tensor([0.1421, 0.1326, 0.1273, 1.2830, 0.2211, 0.0525],
       grad_fn=<AddBackward0>)

In [76]:
net.forward(develop_dataset[3][0])

tensor([0.5387, 0.6924, 1.9000, 0.4925, 0.6030, 0.7858],
       grad_fn=<AddBackward0>)

In [77]:
torch.save(net.state_dict(), "/Users/michaelhein/Programs/Dicebot/models/aggressive_high_lvl_stat.obj")